In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from datetime import datetime, timedelta
import plotly.graph_objects as go

import matplotlib.dates as mdates
import matplotlib.patches as mpatches
from matplotlib.patches import Patch

from plotly.subplots import make_subplots


In [50]:
# bring in data

arkx_data = pd.read_csv(f'../../../data/raw/private_sector/yahoo_finance/arkx_stock_data.csv')
avav_data = pd.read_csv(f'../../../data/raw/private_sector/yahoo_finance/avav_stock_data.csv')
ktos_data = pd.read_csv(f'../../../data/raw/private_sector/yahoo_finance/ktos_stock_data.csv')
trmb_data = pd.read_csv(f'../../../data/raw/private_sector/yahoo_finance/trmb_stock_data.csv')


display(arkx_data.shape)
display(arkx_data.head())


(703, 7)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-03-30,20.500000,20.700001,20.073999,20.299999,20.299999,14594100
1,2021-03-31,20.430000,20.570000,20.410000,20.570000,20.570000,7522400
2,2021-04-01,20.889999,21.000000,20.730000,20.910000,20.910000,6543000
3,2021-04-05,21.270000,21.270000,21.010000,21.139999,21.139999,5038100
4,2021-04-06,21.129999,21.139999,21.030001,21.100000,21.100000,3597900


In [51]:

# convert date
arkx_data['Date'] = pd.to_datetime(arkx_data['Date'])

# create plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=arkx_data['Date'], y=arkx_data['Adj Close'], mode='lines', name='Adj Close', line=dict(color='#ff4500')))

# format
fig.update_layout(title='ARKX Adjusted Close Over Time',
                  xaxis_title='Year',
                  yaxis_title='Adjusted Close',
                  template='plotly_white',
                  width=700, 
                  height=550)



# save and show
fig.write_html('../../docs_source/images/results/arkx_adj_close_plot.html')

fig.show()


/Users/victor/miniconda3/envs/dsan5600/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



## Candlestick Charts

In [52]:

stocks_to_plot = {
    "arkx": arkx_data,
    "avav": avav_data,
    "ktos": ktos_data,
    "trmb": trmb_data,
}


def make_candlestick_plot(stock_data, stock_name):

    # make datetime
    stock_data['Date'] = pd.to_datetime(stock_data['Date'])

    


    # set date as index
    stock_data.set_index('Date', inplace=True)

    # calc SMAs
    stock_data['5_SMA'] = stock_data['Adj Close'].rolling(window=5).mean()
    stock_data['20_SMA'] = stock_data['Adj Close'].rolling(window=20).mean()
    stock_data['100_SMA'] = stock_data['Adj Close'].rolling(window=100).mean()



    # only include data from where 100 sma has vals
    filtered_data = stock_data.loc[stock_data['100_SMA'].first_valid_index():]

    # restrict to one month of data
    one_month_ago = filtered_data.index.max() - pd.DateOffset(months=1)
    filtered_data = filtered_data[filtered_data.index >= one_month_ago ]
    
    #  candelstick chart
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # traces
    fig.add_trace(go.Candlestick(x=filtered_data.index,
                    open=filtered_data['Open'],
                    high=filtered_data['High'],
                    low=filtered_data['Low'],
                    close=filtered_data['Close'], name='Market Data'))
    fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['5_SMA'], mode='lines', name='5 SMA', line=dict(color='blue', width=2)), secondary_y=True)
    fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['20_SMA'], mode='lines', name='20 SMA', line=dict(color='orange', width=2)), secondary_y=True)
    fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['100_SMA'], mode='lines', name='100 SMA', line=dict(color='purple', width=2)), secondary_y=True)


    # format
    fig.update_layout(
        title=f'{stock_name.upper()} Weekly Trading Chart with SMAs',
        yaxis_title='Adjusted Close',
        xaxis_title='Date',
        height=550,
        width=650,
        xaxis_rangeslider_visible=False,  # Hides the range slider at the bottom
        xaxis=dict(
            #rangeselector=dict(
            #    buttons=list([
            #        dict(count=1, label='1M', step='month', stepmode='backward'),
            #        dict(count=6, label='6M', step='month', stepmode='backward'),
            #        dict(count=1, label='YTD', step='year', stepmode='todate'),
            #        dict(count=1, label='1Y', step='year', stepmode='backward'),
            #        dict(step='all')
            #    ])
            #),
            type='date'
        ),
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=-0.3,  # Adjust this value to move the legend up or down
            xanchor='center',
            x=0.5  # Centers the legend horizontally
        )
    )



    # save and show
    fig.write_html(f'../../docs_source/images/results/{stock_name}_crossover_plot_with_candlesticks.html')

    fig.show()


for stock_key, stock_value in stocks_to_plot.items():
    
    make_candlestick_plot(stock_value, stock_key)


